# <center> **Base de données (Films)**

## **Présentation**

Des fichiers csv contenant les données ont été générés lors de la phase précédente de scrapping, nous allons maintenant lire ces fichiers csv dans des dataFrame Pandas et nettoyer les données puis les insérer dans une base de données SQL.


## **Questions**

**Quelles données mettre en SQL et en NoSQL ?**<br>
SQL pour les données tabulaires, NoSQL pour les données non tabulaires ou non structurées.

**Générer automatiquement des IDs chaines de caractères ?**


## **Sources**

**Neo4j pour créer des graphs ML**<br>
https://neo4j.com/docs/getting-started/appendix/tutorials/guide-import-relational-and-etl/


## **Création du schéma relationnel**

![relation_schema](images/DB_diagram.png)

Online tool: https://drawdb.vercel.app/editor<br>

L'utilitaire en ligne **drawdb** permet d'exporter un fichier .SQL contenant la description de notre base (tables avec les relations)

![export_SQL](images/export_SQL.png)

Après quelques modifications à ce fichier nous pouvons créer notre base de données ainsi que les tables à partir de l'invite de commandes.

<code>"C:\Program Files\MySQL\MySQL Server 8.0\bin\mysql.exe" < movies.sql -u root -p</code>



Une difficulté rencontrée est de faire apparaître les films similaires à un film dans la base de données, en effet cette situation est une relation "Many to many" où la table se référence elle-même, c'est un cas fréquent dans les réseaux sociaux où un utilisateur va avoir des amis eux-mêmes utilisateurs.<br>
Deux situations sont observées : <br>
- les relations symétriques (l'utilisateur Alice est amis avec l'utilisateur Bob implique nécessairement que Bob est ami d'Alice),
- les relations asymétriques (Alice est amis avec Bob mais Bob n'est pas forcément ami avec Alice).

Dans notre situation la relation est .....????

Pour représenter cette relation ....





**About Self many-to-many relationship**: https://stackoverflow.com/questions/17128472/many-to-many-on-same-table

Relations n-n

Remarque :
La table "reviews" sert de table de jonction entre les films et les utilisateurs, en effet la relation movies-users est une relation n-n car un utilateur peut écrire des avis pour plusieurs films et un film possède des avis de plusieurs utilisateurs.




## **Création de la base de données**

On lance **MySQL Shell** puis on passe en mode **SQ** avec l'instruction <code>\sql</code>

On peut ensuite lancer la création de notre base de données et des tables en lançant le script "movies.sql"

<code> "C:\Program Files\MySQL\MySQL Server 8.0\bin\mysql.exe" < movies.sql -u root -p</code><br>

Une fois la base 'movies' créée, on peut créer un connecteur sur la base.

In [49]:
import mysql.connector

cnx = mysql.connector.connect(user='root', password='admin', \
                              host = '127.0.0.1', database='movies')
cursor = cnx.cursor(buffered=True)

## **Remplissage de la base de données**

### **Imports**

In [9]:
import math
import copy
import time
import re
import uuid
import numpy as np
import pandas as pd
from tqdm import tqdm
from IPython.display import display

pd.set_option('display.max_rows', 10)
tqdm.pandas()

In [ ]:
cnx.disconnect()

### **Lecture des données à partir des CSV**

In [46]:
# ds_categories = pd.read_csv('csv/categories.csv', delimiter = ',')
# ds_countries = pd.read_csv('csv/countries.csv', delimiter = ',')
arr_categories = np.load('csv/categories.npy')
arr_countries = np.load('csv/countries.npy')
df_movies = pd.read_csv('csv/movies_year_1980.csv', delimiter = ',')

print("Categories :", arr_categories.shape)
print("Countries :", arr_countries.shape)
print("movies :", df_movies.shape)

Categories : (37,)
Countries : (100,)
movies : (109, 16)


### **Remplissage de la table de catégories**

In [ ]:
# To run ONLY ONCE, otherwise : "ERROR Duplicate entry '0' for key 'categories.PRIMARY'"
dict_category_id = {}
for item in arr_categories:
    dict_category_id[item] = str(uuid.uuid4())
    sql = "INSERT INTO categories(category_id, category) VALUES (%s, %s)"
    val = (dict_category_id[item], item)
    cursor.execute(sql, val)
cnx.commit()

### **Remplissage de la table de pays**

In [54]:
# To run ONLY ONCE, otherwise : "ERROR Duplicate entry '0' for key 'countries.PRIMARY'"
dict_country_id = {}
for item in arr_categories:
    dict_country_id[item] = str(uuid.uuid4())
    sql = "INSERT INTO countries(country_id, country) VALUES (%s, %s)"
    val = (dict_country_id[item], item)
    cursor.execute(sql, val)
cnx.commit()

### **Remplissage de la table des producteurs**

In [ ]:
arr_directors = df_movies['directors']

def string_to_list(st):
    ''' Split string into list of strings '''
    return [x for x in st[1:-1].split("'") if len(x) > 1 and x != ', ']

ds_director = df_movies['directors']
ds_director = ds_director.apply(string_to_list)
ds_director = ds_director.apply(pd.Series).stack().reset_index(drop=True)

arr_director = []
for item in ds_director.unique():
    if item.lower() not in arr_director:
        arr_director.append(item.lower())

print("Nb directors : ", len(arr_director))

In [ ]:
dict_director_id = {}
for item in arr_categories:
    dict_director_id[item] = str(uuid.uuid4())
    sql = "INSERT INTO countries(director_id, director) VALUES (%s, %s)"
    val = (dict_director_id[item], item)
    cursor.execute(sql, val)
cnx.commit()

### **Remplissage de la table des acteurs**

### **Remplissage de la table des films**

In [77]:
# --------------------------------------------------------
#
# Converting some columns of "df_movies" into the appropriate format:
#   - correct date format
#   - duration un minutes
#   - IDs of the categories
#   - IDs of the directors / actors / composers
#
# --------------------------------------------------------

def string_list_to_list_of_strings(st):
    st = st.replace(',', '')
    return [item.strip() for item in st[1:-1].split("'") if len(item.strip()) > 1]

txt = "['Anthony Hopkins', 'John Hurt', 'Jack Nicholson', 'Shelley Duvall']"
# txt = "['David Lynch']"
print(string_list_to_list_of_strings(txt))

df_movies_formatted = df_movies.copy()

df_movies_formatted['categories']


df_movies_formatted.head(5)

['Anthony Hopkins', 'John Hurt', 'Jack Nicholson', 'Shelley Duvall']


In [57]:
df_movies.head(5)

,Unnamed: 0,title,date,duration,categories,countries,star_rating,notes,critics,directors,actors,composers,summary,url_thumbnail,url_reviews,url_similar_movies
0,0,Elephant Man,8 avril 1981,2h 05min,"Biopic,Drame",U.S.A.,"4,4",29356,689,['David Lynch'],"['Anthony Hopkins', 'John Hurt', 'Anne Bancrof...",['John Morris (II)'],"Londres, 1884. Le chirurgien Frederick Treves ...",https://fr.web.img4.acsta.net/c_310_420/pictur...,https://www.allocine.fr/film/fichefilm-180/cri...,https://www.allocine.fr/film/fichefilm-180/sim...
1,1,Cannibal Holocaust,22 avril 1981,1h 26min,Epouvante-horreur,"Italie,Colombie","2,1",5795,548,['Ruggero Deodato'],"['Robert Kerman', 'Francesca Ciardi', 'Perry P...",['Riz Ortolani'],Une équipe de journalistes composée de trois h...,https://fr.web.img3.acsta.net/c_310_420/medias...,https://www.allocine.fr/film/fichefilm-52677/c...,https://www.allocine.fr/film/fichefilm-52677/s...
2,2,Shining,16 octobre 1980,2h 23min,"Epouvante-horreur,Thriller","Grande-Bretagne,U.S.A.","4,3",59203,1753,['Stanley Kubrick'],"['Jack Nicholson', 'Shelley Duvall', 'Danny Ll...","['Wendy Carlos', 'Rachel Elkind']","Écrivain, Jack Torrance est engagé comme gardi...",https://fr.web.img3.acsta.net/c_310_420/pictur...,https://www.allocine.fr/film/fichefilm-863/cri...,https://www.allocine.fr/film/fichefilm-863/sim...
3,3,La Boum,17 décembre 1980,1h 49min,"Comédie,Drame,Romance",France,"3,0",18586,227,['Claude Pinoteau'],"['Sophie Marceau', 'Brigitte Fossey', 'Claude ...",['Vladimir Cosma'],"Vic vit tranquillement entre le lycée, ses par...",https://fr.web.img3.acsta.net/c_310_420/medias...,https://www.allocine.fr/film/fichefilm-4403/cr...,https://www.allocine.fr/film/fichefilm-4403/si...
4,4,Beau-Père,16 septembre 1981,2h 03min,Drame,France,"3,6",821,82,['Bertrand Blier'],"['Patrick Dewaere', 'Ariel Besse', 'Maurice Ro...",['Philippe Sarde'],"Rémi est un musicien à la dérive. Sa compagne,...",https://fr.web.img3.acsta.net/c_310_420/pictur...,https://www.allocine.fr/film/fichefilm-1403/cr...,https://www.allocine.fr/film/fichefilm-1403/si...


In [61]:
print(type(df_movies.loc[0, 'categories']))
df_movies.loc[0, 'categories']

<class 'str'>


'Biopic,Drame'